### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [3]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [4]:
# cell for work
#'The number of movies in the dataset'
print( "The number of movies is {}.".format(movies.shape[0]))

The number of movies is 35479.


In [5]:
# cell for work
#'The number of ratings in the dataset'
print("The number of ratings is {}.".format(reviews.shape[0]))

The number of ratings is 863866.


In [6]:
#'The number of different genres'

movies['genre'].value_counts()

genere_categories = movies['genre'].str.split(pat = '|' , expand = True)
ab = genere_categories[0].value_counts()
ab.count()

28

In [7]:
# number of different genres/ Another way
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))

The number of genres is 28.


In [8]:
# more work
#'The number of unique users in the dataset':
print("The number of unique users is {}.".format(reviews.user_id.nunique()))

The number of unique users is 67353.


In [7]:
# and still more
#'The number missing ratings in the reviews dataset':
int(reviews.rating.isnull().mean()*reviews.shape[0])

0

In [8]:
# and even more - open additional cells as necessary


In [9]:
# Use your findings to match each variable to the correct statement in the dictionary
a = reviews.user_id.nunique()
b = reviews.rating.max()
c = np.round(reviews.rating.mean(), 0)
d = movies.shape[0]
e = reviews.rating.min()
f = int(reviews.rating.isnull().mean()*reviews.shape[0])
h = reviews.shape[0]
i = len(genres)

dict_sol1 = {
'The number of movies in the dataset': d, 
'The number of ratings in the dataset': h,
'The number of different genres': i, 
'The number of unique users in the dataset': a, 
'The number missing ratings in the reviews dataset': f, 
'The average rating given across all ratings': c,
'The minimum rating given across all ratings': e,
'The maximum rating given across all ratings': b
}

# Check your solution
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [10]:
# cell for work
print(movies.head())
movies['movie'].head()

  movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  
0  Documentary|Short  
1  Documentary|Short  
2  Documentary|Short  
3                NaN  
4       Short|Horror  


0        Edison Kinetoscopic Record of a Sneeze (1894)
1                  La sortie des usines Lumière (1895)
2                        The Arrival of a Train (1896)
3    The Oxford and Cambridge University Boat Race ...
4                           Le manoir du diable (1896)
Name: movie, dtype: object

In [11]:
#Extract Movie date
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

movies['date'] = movies['movie'].apply(create_date)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [12]:
#Create Dummy value for century
century_1800s = lambda n : 1 if n[:-2] == '18' else 0
century_1900s = lambda n1 : 1 if n1[:-2] == '19' else 0
century_2000s = lambda n2 : 1 if n2[:-2] == '20' else 0

movies['1800s'] = movies['date'].apply(century_1800s)
movies['1900s'] = movies['date'].apply(century_1900s)
movies['2000s'] = movies['date'].apply(century_2000s)

In [13]:
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

In [14]:
movies

,movie_id,movie,genre,date,1800s,1900s,2000s,Music,Film-Noir,Thriller,...,News,Game-Show,War,Documentary,Reality-TV,Biography,Mystery,Talk-Show,Sci-Fi,Fantasy
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0000131,Une nuit terrible (1896),Short|Comedy|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0000417,A Trip to the Moon (1902),Short|Action|Adventure|Comedy|Fantasy|Sci-Fi,1902,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
7,0000439,The Great Train Robbery (1903),Short|Action|Crime|Western,1903,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0443,"Hiawatha, the Messiah of the Ojibway (1903)",NaN,1903,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0000628,The Adventures of Dollie (1908),Action|Short,1908,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

reviews['date'] = reviews['timestamp'].apply(change_timestamp)

In [17]:
reviews['date']

0         2013-10-05 21:00:50
1         2020-04-09 21:01:12
2         2020-01-15 03:10:27
3         2020-01-09 20:50:53
4         2020-01-13 22:48:17
5         2020-01-20 22:27:24
6         2020-03-08 22:25:05
7         2020-01-25 10:57:22
8         2020-04-09 12:45:54
9         2020-01-17 11:50:30
10        2020-03-15 19:20:27
11        2014-01-31 22:27:59
12        2014-01-05 20:57:18
13        2013-11-02 19:15:33
14        2013-12-27 18:46:47
15        2015-01-30 21:13:47
16        2013-04-12 03:46:57
17        2013-05-10 22:23:19
18        2013-08-23 02:39:49
19        2013-04-12 03:48:24
20        2013-08-23 02:41:26
21        2015-05-04 00:35:54
22        2015-02-02 06:38:23
23        2016-05-11 16:05:25
24        2016-05-10 09:54:43
25        2014-01-17 23:35:06
26        2015-12-10 21:48:29
27        2015-03-01 19:45:29
28        2016-08-30 22:49:49
29        2016-08-30 21:14:15
                 ...         
863836    2020-03-22 12:44:42
863837    2020-01-11 21:50:57
863838    

In [19]:
# now reviews and movies are the final dataframes with the necessary columns
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')

In [18]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year

In [74]:
reviews_new

,Unnamed: 0,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,5,2,816711,8,1379963769,2013-09-23 19:16:09,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,6,2,1091191,7,1391173869,2014-01-31 13:11:09,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,7,2,1103275,7,1408192129,2014-08-16 12:28:49,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,8,2,1322269,7,1391529691,2014-02-04 16:01:31,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,9,2,1390411,8,1451374513,2015-12-29 07:35:13,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
